# Audio Fingerprinting

This notebook demonstrates the complete audio fingerprinting pipeline:
1. Spectrogram computation (STFT)
2. Local-maxima extraction
3. Target-zone hashing
4. Database construction and song identification

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

# Import the standalone fingerprinting module
from Faycal_Raghibi_audio_fingerprinting import (
    compute_spectrogram,
    get_maxima,
    process_signal,
    AudioDatabase,
)

## 1. Load an Audio File

We load one of the provided classical music excerpts at the required sampling rate of 3000 Hz.

In [ ]:
SONGS_DIR = "../songs"
song_files = [f for f in os.listdir(SONGS_DIR) if f.endswith(".mp3")]
print(f"Available songs ({len(song_files)}):")
for s in song_files:
    print(f"  • {s}")

# Pick the first song for demonstration
SONG_PATH = os.path.join(SONGS_DIR, song_files[0])
y, sr = librosa.load(SONG_PATH, sr=3000)
print(f"\nLoaded: {song_files[0]}")
print(f"  Duration : {len(y)/sr:.1f} s")
print(f"  Samples  : {len(y)}")
print(f"  SR       : {sr} Hz")

## 2. Spectrogram Computation

The spectrogram is computed using the Short-Time Fourier Transform (STFT) with the following parameters:
- `n_fft = 2048`
- `hop_length = 512`
- `win_length = 1024`

We obtain the **energy spectrogram** $|V_g f(u, \xi)|^2$.

In [ ]:
S = compute_spectrogram(y, sr=sr)
print(f"Spectrogram shape: {S.shape}  (freq_bins x time_frames)")

plt.figure(figsize=(14, 5))
librosa.display.specshow(
    librosa.power_to_db(S, ref=np.max),
    sr=sr, hop_length=512, x_axis="time", y_axis="hz"
)
plt.colorbar(format="%+2.0f dB")
plt.title("Energy Spectrogram")
plt.tight_layout()
plt.show()

## 3. Local Maxima (Constellation Map)

We identify spectral peaks by finding local maxima in 3×3 grids, then filter by:
- Amplitude threshold (90th percentile)
- Minimum distance pruning (10 px)

In [ ]:
maxima = get_maxima(S)
print(f"Detected {len(maxima)} spectral peaks")

plt.figure(figsize=(14, 5))
librosa.display.specshow(
    librosa.power_to_db(S, ref=np.max),
    sr=sr, hop_length=512, x_axis="time", y_axis="hz",
    alpha=0.6
)
plt.scatter(
    maxima[:, 1] * 512 / sr,  # convert frames to seconds
    maxima[:, 0] * sr / 2048, # convert bins to Hz
    c="red", s=5, marker=".", label="Peaks"
)
plt.colorbar(format="%+2.0f dB")
plt.title("Constellation Map (spectral peaks overlaid)")
plt.legend()
plt.tight_layout()
plt.show()

## 4. Hash Generation

For each anchor peak, we define a **Target Zone** (300 px in time, 20 px in frequency) and pair the anchor with the top-10 peaks inside the zone.

$$\text{Hash} = f_{\text{anchor}} \times 10^6 + f_{\text{target}} \times 10^3 + (t_{\text{target}} - t_{\text{anchor}})$$

In [ ]:
hashes = process_signal(y, sr=sr)
print(f"Generated {len(hashes)} unique hashes")
print("\nSample hashes (first 10):")
for i, (h, t) in enumerate(list(hashes.items())[:10]):
    print(f"  Hash: {h:>15d}   Anchor time frame: {t}")

## 5. Database Construction

We fingerprint every song in the `songs/` directory and store the resulting hash dictionaries in a pickle file.

In [ ]:
DB_PATH = "../dataset/dataset.pickle"
db = AudioDatabase(db_path=DB_PATH)

if not os.path.exists(DB_PATH):
    print("Building database (this may take a few minutes)...")
    db.create_database(songs_dir=SONGS_DIR)
else:
    db.load_database()

print(f"\nDatabase contains {len(db.database)} songs:")
for name in db.song_names:
    print(f"  • {name}")

## 6. Song Identification

We simulate a real-world query by taking a 10-second excerpt from one of the stored songs, fingerprinting it, and searching the database.

In [ ]:
# Create a 10-second excerpt from the middle of the song
excerpt_start = len(y) // 3
excerpt_len = 10 * sr  # 10 seconds
excerpt = y[excerpt_start : excerpt_start + excerpt_len]
print(f"Excerpt: {len(excerpt)/sr:.1f} s  (frames {excerpt_start}–{excerpt_start+excerpt_len})")

# Fingerprint the excerpt
excerpt_hashes = process_signal(excerpt, sr=sr)
print(f"Excerpt hashes: {len(excerpt_hashes)}")

# Search
from Faycal_Raghibi_audio_fingerprinting import search_song

top_indices = search_song(db.database, excerpt_hashes)
print("\n--- Top 3 Matches ---")
for rank, idx in enumerate(top_indices, 1):
    print(f"  {rank}. {db.song_names[idx]}")

## 7. Summary

| Step | Description | Output |
|------|-------------|--------|
| 1 | Load audio at 3 kHz | Mono signal |
| 2 | STFT (n_fft=2048, hop=512, win=1024) | Energy spectrogram |
| 3 | 3×3 local-max + filtering | Constellation map |
| 4 | Target-zone pairing + hashing | Dict of hashes |
| 5 | Repeat for all songs → pickle DB | Persistent database |
| 6 | Hash an excerpt, histogram match | Identified song |

The system correctly identifies the source song from a short excerpt by matching the histogram of time-offset differences between shared hashes.